In [ ]:
import time
start_time = time.time()
%matplotlib qt
mpl_time = time.time()
import pyoptflight as pof
pof_main_time = time.time()
from pyoptflight import initialize as optinit
pof_init_time = time.time()
from pyoptflight import plotting as optplot
pof_plotting_time = time.time()
import numpy as np
np_time = time.time()

print(f'mpl: {mpl_time - start_time}')
print(f'pof main: {pof_main_time - mpl_time}')
print(f'pof init: {pof_init_time - pof_main_time}')
print(f'pof plotting: {pof_plotting_time - pof_init_time}')
print(f'np: {np_time - pof_plotting_time}')

In [ ]:
kerbin = pof.Body("Kerbin")
kerbin.omega_0 = 0
vehicle = pof.Stage.load_vehicle('mintoc_multi')
# vehicle = pof.Stage.load_vehicle('mintoc_single')

vehicle[0].m_f = 8.4
vehicle[1].m_0 = 8.3

config = pof.SolverConfig(landing=False, 
                          T_min      = 1,
                          T_max      = 700,
                          max_iter   = 10000, 
                          solver_tol = 1e-3, 
                          N          = 50, 
                          T_init     = 100,
                          integration_method = 'RK4')

x0 = pof.LatLngBound(lat=0, lng=0, alt=0, vel = 1e-6, ERA0=0)

xf = pof.KeplerianBound(i    = np.deg2rad(20),
                        Ω    = np.deg2rad(0),
                        ω    = 0,
                        ha   = 120,
                        hp   = 120,
                        body = kerbin)

msolver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [ ]:
msolver.update_constraints(constraint_names='max_tau', new_values=0.10, new_enables=True)
msolver.update_constraints(constraint_names='max_body_rate_y', new_values=np.deg2rad(5), new_enables=True)
msolver.update_constraints(constraint_names='max_body_rate_z', new_values=np.deg2rad(5), new_enables=True)
msolver.update_constraints(constraint_names='f_min', new_values=0.15, new_enables=True)
msolver.update_constraints(constraint_names='max_alpha', new_values=np.deg2rad(10), new_enables=True)
msolver.update_constraints(constraint_names='max_q', new_values=0.2, new_enables=True)
msolver.constraints

In [ ]:
msolver.create_nlp()

In [ ]:
msolver.initialize_from_func(pof.gravity_turn, opts={'skew':True})

In [ ]:
pof.plot_solutions(msolver, colorscale=None, markers=None, show_actual_orbit=True, show_target_orbit=True, indices=[-1])

In [ ]:
pof.plot_flight_data(msolver)